## Ingest results.json file

1. Read the JSON file using the spark dataframe reader

In [0]:
from pyspark.sql.functions import col, lit

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
result_schema = "resultId INT, raceId INT, driverId INT, constructorId INT, number INT, grid INT, position INT, positionText STRING, positionOrder INT, points FLOAT, laps INT, time STRING, milliseconds INT, fastestLap INT, rank INT, fastestLapTime STRING, fastestLapSpeed STRING, statusId INT"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
result_df = spark.read \
  .schema(result_schema) \
  .json(f'{raw_folder_path}/results.json')
#display(result_df)

2. Process Dataframe

In [0]:
result_final_df = result_df.withColumnRenamed("resultId", "result_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("DriverId", "driver_id") \
    .withColumnRenamed("constructorId", "constructor_id") \
    .withColumnRenamed("positionText", "position_text") \
    .withColumnRenamed("positionOrder", "position_order") \
    .withColumnRenamed("fastestLap", "fastest_lap") \
    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
    .drop(col('statusId')) \
    .withColumn("data_source", lit(v_data_source))
result_final_df = add_ingestion_date(result_final_df)
#display(result_final_df)

3. Write data to datalake as parquet

In [0]:
#result_final_df.write.mode("overwrite").partitionBy('race_id').parquet(f"{processed_folder_path}/results")
# database
result_final_df.write.mode("overwrite").partitionBy('race_id').format("parquet").saveAsTable("f1_processed.results")

In [0]:
# %fs
# ls mnt/formula1jf/processed/results

In [0]:
dbutils.notebook.exit("Success")